<a href="https://colab.research.google.com/github/njonge-nathan/CAT02_MINI_PROJECT/blob/main/CAT02_MINI_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Student Number 094230

CAT 2 - MINI PROJECT

COMPILER CONSTRUCTION



## Question

 CAT 2 is a mini project. Implement any phase, component, element etc learned in the course thus far.

Ideas can range from implementing a lexer, parser to generating your own language (i.e. CFG) etc.

Whatever you implement make sure to describe your solution in detail by outlining its logic, concepts used among other relevant details.

Marks will be awarded based on:

Uniqueness of idea (3 marks)

Description of details and logic used (6 marks)

A show of connection between the solution and the concept(s) learned in class (4 marks)

Proper formatting and editing of the implementation notebook (2 marks)
Total = 15 marks.

## Solution Implementation

The solution involves an implementation of a simple compiler construction solution that includes both lexical analysis (using a lexer) and syntactic analysis (using a parser) for a basic arithmetic expression language. We'll use Python for this example.

### Lexer Implementation

In [3]:
import re

class Lexer:
    """
    Lexer class for tokenizing arithmetic expressions.
    """

    def __init__(self, expression):
        """
        Initialize the Lexer with the input expression.

        Parameters:
        - expression (str): The arithmetic expression to tokenize.
        """
        self.expression = expression
        self.tokens = []
        self.current_position = 0

    def tokenize(self):
        """
        Tokenize the input expression.

        Returns:
        - list: List of tokens representing the arithmetic expression.
        """
        while self.current_position < len(self.expression):
            char = self.expression[self.current_position]

            if char.isspace():
                # Skip whitespace
                self.current_position += 1
                continue
            elif char.isdigit():
                # Tokenize numbers
                self.tokens.append(self.tokenize_number())
            elif char in {'+', '-', '*', '/'}:
                # Tokenize operators
                self.tokens.append({'type': 'OPERATOR', 'value': char})
                self.current_position += 1
            elif char == '(' or char == ')':
                # Tokenize parentheses
                self.tokens.append({'type': 'PARENTHESIS', 'value': char})
                self.current_position += 1
            else:
                raise ValueError(f"Invalid character: {char}")

        return self.tokens

    def tokenize_number(self):
        """
        Tokenize a sequence of digits as a number.

        Returns:
        - dict: Token representing a number.
        """
        number = ''
        while self.current_position < len(self.expression) and self.expression[self.current_position].isdigit():
            number += self.expression[self.current_position]
            self.current_position += 1

        return {'type': 'NUMBER', 'value': int(number)}


#### Description

Lexer: Tokenizes the input expression into a sequence of tokens. The Lexer class uses regular expressions to recognize numbers and operators.

### Parser Implementation

In [4]:
class Parser:
    """
    Parser class for parsing arithmetic expressions.
    """

    def __init__(self, tokens):
        """
        Initialize the Parser with a list of tokens.

        Parameters:
        - tokens (list): List of tokens representing the arithmetic expression.
        """
        self.tokens = tokens
        self.current_position = 0

    def parse(self):
        """
        Parse the arithmetic expression.

        Returns:
        - dict: Abstract syntax tree (AST) representing the expression structure.
        """
        return self.parse_expression()

    def parse_expression(self):
        """
        Parse terms and handle addition/subtraction.

        Returns:
        - dict: AST node representing the expression.
        """
        left_operand = self.parse_term()

        while self.current_position < len(self.tokens) and self.tokens[self.current_position]['type'] == 'OPERATOR' and self.tokens[self.current_position]['value'] in {'+', '-'}:
            operator = self.tokens[self.current_position]['value']
            self.current_position += 1
            right_operand = self.parse_term()

            left_operand = {'type': 'EXPRESSION', 'operator': operator, 'left': left_operand, 'right': right_operand}

        return left_operand

    def parse_term(self):
        """
        Parse factors and handle multiplication/division.

        Returns:
        - dict: AST node representing the term.
        """
        factor = self.parse_factor()

        while self.current_position < len(self.tokens) and self.tokens[self.current_position]['type'] == 'OPERATOR' and self.tokens[self.current_position]['value'] in {'*', '/'}:
            operator = self.tokens[self.current_position]['value']
            self.current_position += 1
            term = self.parse_factor()

            factor = {'type': 'TERM', 'operator': operator, 'left': factor, 'right': term}

        return factor

    def parse_factor(self):
        """
        Parse factors including numbers and expressions in parentheses.

        Returns:
        - dict: AST node representing the factor.
        """
        if self.current_position < len(self.tokens):
            current_token = self.tokens[self.current_position]

            if current_token['type'] == 'NUMBER':
                # If it's a number, consume the token and return it
                self.current_position += 1
                return current_token
            elif current_token['type'] == 'PARENTHESIS' and current_token['value'] == '(':
                # If it's an opening parenthesis, consume it, parse the expression inside, and check for a closing parenthesis
                self.current_position += 1
                expression = self.parse_expression()
                if self.current_position < len(self.tokens) and self.tokens[self.current_position]['type'] == 'PARENTHESIS' and self.tokens[self.current_position]['value'] == ')':
                    self.current_position += 1
                    return expression

        raise ValueError("Invalid expression syntax")


#### Description

Parser: Parses the sequence of tokens into an abstract syntax tree (AST) that represents the structure of the expression.

Parser Methods: The parser has methods for parsing expressions, terms, and factors. It follows the grammar rules for arithmetic expressions, considering operator precedence and parentheses.

AST Representation: The AST is represented using nested dictionaries, where each node has a type ('EXPRESSION', 'TERM', 'NUMBER') and relevant values.


The provided Python code for the lexer and parser is designed to tokenize and parse arithmetic expressions, building an Abstract Syntax Tree (AST) to represent the structure of the expression.

In [5]:
def print_ast(node, indent=""):
    if 'value' in node:
        print(f"{indent}Type: {node['type']}, Value: {node['value']}")
    else:
        print(f"{indent}Type: {node['type']}")
        print(f"{indent}Operator: {node['operator']}")
        print_ast(node['left'], indent + "  ")
        print_ast(node['right'], indent + "  ")

# Example Usage
expression = "3 + 4 * (2 - 7) / 3"
lexer = Lexer(expression)
tokens = lexer.tokenize()

parser = Parser(tokens)
parsed_tree = parser.parse()

print_ast(parsed_tree)


Type: EXPRESSION
Operator: +
  Type: NUMBER, Value: 3
  Type: TERM
  Operator: /
    Type: TERM
    Operator: *
      Type: NUMBER, Value: 4
      Type: EXPRESSION
      Operator: -
        Type: NUMBER, Value: 2
        Type: NUMBER, Value: 7
    Type: NUMBER, Value: 3


This print_ast function recursively traverses the AST and prints information about each node, indicating the type and value for leaf nodes (numbers) and the type and operator for internal nodes.

For the provided example expression ("3 + 4 * (2 - 7) / 3"), the output of the print_ast

This implementation connects to the concepts of compiler construction, specifically lexical and syntactic analysis.

Lexer (Lexical Analysis): Tokenization is a crucial step in the lexical analysis phase. The lexer recognizes and categorizes characters into meaningful tokens.

Parser (Syntactic Analysis): The parser processes the tokens and builds a hierarchical representation (AST) that reflects the syntactic structure of the input expression.